##### The dataset we are working on consists of 1376 images with 690 images containing images of people wearing masks and 686 images with people without masks.

## 1. Import all the libraries and modules required.

In [3]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

## 2. Build the neural network:

This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset. Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.

And then two Dense layers for classification.

In [4]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## 3. Image Data Generation/Augmentation:

In [5]:
TRAINING_DIR = "dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


## 4. Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [6]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

## 5. Train the model:

In [7]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

F:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
132/132 [==============================] - 121s 817ms/step - loss: 0.6206 - acc: 0.6796 - val_loss: 0.1363 - val_acc: 0.9536
INFO:tensorflow:Assets written to: model2-001.model\assets
Epoch 2/10
132/132 [==============================] - 72s 548ms/step - loss: 0.2657 - acc: 0.9033 - val_loss: 0.1342 - val_acc: 0.9691
INFO:tensorflow:Assets written to: model2-002.model\assets
Epoch 3/10
132/132 [==============================] - 71s 541ms/step - loss: 0.2922 - acc: 0.8842 - val_loss: 0.1719 - val_acc: 0.9227
Epoch 4/10
132/132 [==============================] - 73s 553ms/step - loss: 0.2678 - acc: 0.8914 - val_loss: 0.1857 - val_acc: 0.8969
Epoch 5/10
132/132 [==============================] - 72s 547ms/step - loss: 0.2410 - acc: 0.9148 - val_loss: 0.0776 - val_acc: 0.9691
INFO:tensorflow:Assets written to: model2-005.model\assets
Epoch 6/10
132/132 [==============================] - 79s 597ms/step - loss: 0.2425 - acc: 0.9142 - val_loss: 0.1136 - val_acc: 0.9485
Epoch 7/10
1

# Now we will test the results of face mask detector model using OpenCV.

In [14]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("model2-005.model/")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(1) 


haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()

cv2.destroyAllWindows()